# Extract the Personal Identifiable Information (PII) using Watson NLP

<h2>Use Case</h2>

This notebook demonstrates how to extract PII entities using Watson NLP Pre-trained  models also demonstrates how to prepare custom train models. PII extraction is the process of identifying and extracting personal information from a document or dataset. This information can include names, addresses, phone numbers, email addresses, Social Security numbers, Credit Card number, and other types of information that can be used to identify an individual. 


<h2>What you'll learn in this notebook</h2>

Watson NLP offers Pre-trained Models for various NLP tasks also provides fine-tune functionality for custom training. This notebooks shows:

* <b>RBR</b>:  A Rule-Based Reasoner (RBR) in NLP works by using a set of predefined rules to process and understand natural language input. These rules are used to identify specific patterns or structures in the input text and determine the meaning of the text based on those patterns.


* <b>BILSTM</b>: the BiLSTM network would take the preprocessed text as input and learn to identify patterns and relationships between words that are indicative of PII data. The BiLSTM network would then output a probability score for each word in the text, indicating the likelihood that the word is part of a PII entity. The BiLSTM network may also be trained to recognize specific entities such as names, addresses, phone numbers, email addresses, etc.


* <b>SIRE</b>: Statistical Information and Relation Extraction (SIRE) is a technique used in natural language processing (NLP) to extract specific information and relationships from text. It involves using machine learning algorithms to identify and extract structured data such as entities, attributes, and relations from unstructured text. SIRE is used in a variety of applications, including information extraction, knowledge graph construction, and question answering. SIRE typically uses supervised learning approach, where a model is trained using annotated examples of text and the corresponding structured data. The model can then be used to extract the same information from new, unseen text.

## Table of Contents


1. [Before you start](#beforeYouStart)
1. [Load Entity PII Models](#LoadModel)
1. [Data generation for Testing](#Loaddata)
1. [Watson NLP Models](#NLPModels)    
   1. [BiLSTM Pretrained](#BILSTMPre)
   1. [RBR Pretrained](#RBRPre)
   1. [SIRE Pretrained](#Sire)
1. [Testing Usecase](#Testing)  
   1.  [RBR Model Testing for Caredit card Extraction, URL and Email](#5.1)
       1. [RBR Stock - URL Extraction](#5.1.1)
       1. [Combine RBR PII and RBR Stock (URL)](#5.1.2)
   1. [EmailAddress Extraction using RBR](#5.2)
1. [Summary](#summary)

<a id="beforeYouStart"></a>
### 1. Before you start


<div class="alert alert-block alert-danger">
<b>Stop kernel of other notebooks.</b></div>

**Note:** If you have other notebooks currently running with the _Default Python 3.x environment, **stop their kernels** before running this notebook. All these notebooks share the same runtime environment, and if they are running in parallel, you may encounter memory issues. To stop the kernel of another notebook, open that notebook, and select _File > Stop Kernel_.

<div class="alert alert-block alert-warning">
<b>Set Project token.</b></div>

Before you can begin working on this notebook in Watson Studio in Cloud Pak for Data as a Service, you need to ensure that the project token is set so that you can access the project assets via the notebook.

When this notebook is added to the project, a project access token should be inserted at the top of the notebook in a code cell. If you do not see the cell above, add the token to the notebook by clicking **More > Insert project token** from the notebook action bar.  By running the inserted hidden code cell, a project object is created that you can use to access project resources.

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

<div class="alert alert-block alert-info">
<b>Tip:</b> Cell execution</div>

Note that you can step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.

In [2]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.9 MB/s eta 0:00:00


In [3]:
import json
import pandas as pd
import watson_nlp
import random
from faker import Faker
from watson_nlp import data_model as dm
from watson_nlp.toolkit.entity_mentions_utils import prepare_train_from_json

In [4]:
# Silence Tensorflow warnings
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

<a id="LoadModel"></a>
### 2. Load Entity PII Models

The function "watson_nlp.load()" is used to download a pre-trained NLP model, such as IBM Watson NLP, which can then be used to process and analyze text data. This function allows you to quickly and easily integrate advanced NLP capabilities into your application, without having to train a model from scratch. By downloading a pre-trained model, you can save time and resources, and benefit from the expertise of the model developers.

In [5]:
# Load a syntax model to split the text into sentences and tokens
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock'))
# Load bilstm model in WatsonNLP
bilstm_model = watson_nlp.load(watson_nlp.download('entity-mentions_bilstm_en_pii'))
# Load rbr model in WatsonNLP
rbr_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_multi_pii'))
# sir Load rbr model in WatsonNLP
sire = watson_nlp.load(watson_nlp.download('entity-mentions_sire_en_stock-wf'))

<a id="Loaddata"></a>
### 3. Data generation for Testing

Generate the data Name, SSN and credit card numbers using faker. 

In [6]:

fake = Faker(locale='en_US')

def format_data():  
        #Generate a random
        name = fake.name() 

        #Generate a random SSN 
        ssn = fake.ssn()

        #Generate a random CCN 
        ccn = fake.credit_card_number()

        text_1 = """My name is %s, and my social security number is %s. Here's the number to my Visa credit card is %s""" % (name, ssn, ccn)

        text_2 = """%s is my social security number. The name on my credit card %s is %s."""% (ssn, ccn, name)

        text_3 = """My credit card number is %s and social security number is %s, I am %s""" %(ccn,ssn,name)


        text = random.choice([text_1, text_2,text_3])
        
        return text

In [7]:
text = format_data()
text

'My credit card number is 4084897820169892970 and social security number is 178-32-7893, I am Theresa Bowen'

<a id="NLPModels"></a>
### 4.  Watson NLP Models

<a id="BILSTMPre"></a>
### 4. 1 BiLSTM Pretrained

The term "pretrained" refers to a pre-trained BiLSTM model, which has already been trained on a large corpus of text data and can be fine-tuned or used as is for specific NLP tasks, such as sentiment analysis, named entity recognition, and so on. By using a pretrained BiLSTM model, you can leverage the knowledge learned from the training data to quickly build NLP applications with improved accuracy.

In [8]:
#help(bilstm_model)

In [9]:
#Test Pretrained bilstm_model model in WatsonNLP
syntax_result = syntax_model.run(text)
bilstm_result = bilstm_model.run(syntax_result)

for i in bilstm_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  Theresa Bowen   Type:  Person


<a id="RBRPre"></a>

### 4.2 RBR Pretrained


A pretrained rule-based model is a model that has already been trained on a large corpus of text data and has a set of predefined rules for processing text data. By using a pretrained rule-based model, you can leverage the knowledge learned from the training data to quickly build NLP applications with improved accuracy.

In [10]:
text1= format_data()
text2= format_data()
text3= format_data()

In [11]:
all_test=[text1,text2,text3]
all_test

['My credit card number is 4913111056139226 and social security number is 396-04-7877, I am Willie Jones',
 "My name is James Johnson, and my social security number is 168-63-3593. Here's the number to my Visa credit card is 180022879960314",
 'My credit card number is 060446370110 and social security number is 891-08-6000, I am Cynthia Baker']

In [12]:
# help(rbr_model)

In [13]:
t=1
# Test the pretrain
for test in all_test:
    rbr_result = rbr_model.run(test, language_code='en')

    for i in rbr_result.mentions:
        print("Text",t, i.span.text.ljust(15, " "), "Type: ", i.type)
    t+=1

Text 1 4913111056139226 Type:  BankAccountNumber.CreditCardNumber.Visa
Text 1 396-04-7877     Type:  NationalNumber.SocialSecurityNumber.US
Text 2 168-63-3593     Type:  NationalNumber.SocialSecurityNumber.US
Text 3 891-08-6000     Type:  NationalNumber.SocialSecurityNumber.US


<a id="Sire"></a>

### 4.3 SIRE Pretrained


In [14]:
#test dataset for URL extraction 
test1 = "My name is Robert Aragon, and my social security number is 489-36-8350. Here's the number to my credit card, http://www.example.com/page_id=5555555555554444"

In [15]:
#Test Pretrained sire_model model in WatsonNLP
sire_result = sire.run(test1)

for i in sire_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)


Text:  Robert Aragon   Type:  Person


<a id="Testing"></a>

## 5. Testing Usecase

<a id="5.1"></a>
### 5.1 RBR Model Testing for Caredit card Extraction, URL and Email

<a id="5.1.1"></a>
#### 5.1.1 RBR Stock (URL Extraction)

In [16]:
#Test Pretrained rbr stock model in WatsonNLP
rbr_ent_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_en_stock'))
rbr_ent_result = rbr_ent_model.run(test1)

for i in rbr_ent_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  PhoneNumber
Text:  http://www.example.com/page_id=5555555555554444 Type:  URL


In [17]:
#Test Pretrained rbr PII model in WatsonNLP

rbr_result_pii = rbr_model.run(test1, language_code='en')
rbr_result_pii

for i in rbr_result_pii.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  NationalNumber.SocialSecurityNumber.US


<a id="5.1.2"></a>
#### 5.1.2 Combine RBR PII and RBR Stock (URL)

In [18]:
# combine to be one object
combined_mentions = rbr_ent_result + rbr_result_pii
combined_mentions

for i in combined_mentions.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  PhoneNumber
Text:  489-36-8350     Type:  NationalNumber.SocialSecurityNumber.US
Text:  http://www.example.com/page_id=5555555555554444 Type:  URL


<a id="5.2"></a>
### 5.2 EmailAddress Extraction using RBR 

In [26]:
#test dataset for EMail extraction 
text = format_data() +" my maild id is sample.Email@gmail.com"
text

'175-43-9027 is my social security number. The name on my credit card 3572267594198019 is Mathew Frank. my maild id is sample.Email@gmail.com'

In [27]:
#Test Pretrained rbr PII model in WatsonNLP

rbr_result_pii = rbr_model.run(text, language_code='en')
rbr_result_pii

for i in rbr_result_pii.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  175-43-9027     Type:  NationalNumber.SocialSecurityNumber.US
Text:  3572267594198019 Type:  BankAccountNumber.CreditCardNumber.Other
Text:  sample.Email@gmail.com Type:  EmailAddress


<a id="summary"></a>
## 6. Summary

<span style="color:blue">This notebook shows you how to use the Watson NLP library to:
1. Extract PII using Pre-trained Models
</span>

Please note that this content is made available by IBM Build Lab to foster Embedded AI technology adoption. The content may include systems & methods pending patent with USPTO and protected under US Patent Laws. For redistribution of this content, IBM will use release process. For any questions please log an issue in the GitHub.

Developed by IBM Build Lab

Copyright - 2023 IBM Corporation